## Background

- Analyzing conversion funnels is crucial for understanding and optimizing the user journey on a website.
- Conversion funnels represent the step-by-step process users go through on a website or an app, from initial interaction to a desired action (e.g., making a purchase or signing up).

**Key Components:**
1. **Entry Points:** Identify where users enter the funnel, such as landing pages or specific campaigns.
2. **User Flow:** Track the sequence of actions users take within the funnel, highlighting drop-off points.
3. **Conversion Goals:** Define specific actions that signify successful conversions, like completing a form or making a purchase.
4. **Metrics:** Utilize key metrics like conversion rate, bounce rate, and average time spent to gauge funnel performance.

**Funnel Types:**
1. **Open Funnel**
- Considers whether an event is represented in a user’s path or not, not necessarily in any defined sequence.
- Changing the sequence of events doesn't impacts the funnel numbers.  
2. **Closed Funnel**
- Stages are defined in a sequential manner, ensuring that the users at each stage have passed all the previous stages, example - a path pattern A → B → C. This means that the users at the first stage of the funnel must contain A event, the users at the second stage must have a sub-path A → B, and finally the users at the third stage must contain A → B → C.
- Changing the sequence of events impacts the funnel numbers.
3. **Hybrid Funnel**
- This funnels is a combination of both open and closed funnels.
- The first stage of a hybrid funnel is formed by the users who experienced A event. That is why the users at this stage are identical to the users of open and closed funnel at this stage.
- Next, for the second stage of the funnel consider only the users who experienced event B after event A. So far the logic is the same as for closed funnel, and that is why the users at the second level of the hybrid funnel are the same as the users at the second level of the closed funnel (2 users).However, at the third stage we can see some differences with the closed funnel.
- Based on the users from the previous stage - amongst these users we select only those who experienced event C after event A, disregarding the fact that whether it happened after B event or not.


**Benefits:**
1. **Optimization Opportunities:** Pinpoint areas for improvement by analyzing where users are dropping off or facing obstacles.
2. **User Experience Enhancement:** Understand user behavior to enhance the overall website experience and streamline the conversion process.
3. **Data-Driven Decision Making:** Make informed decisions based on data, optimizing marketing strategies and website design for better results.


Effective analysis of conversion funnels is vital for businesses seeking to maximize user engagement and achieve higher conversion rates. Regular monitoring and optimization based on analytics insights lead to an improved user experience.

# 1. Loading data

In [1]:
# import required libraries
from retentioneering import datasets

In [2]:
# load sample user behavior data:
stream = datasets.load_simple_shop()

In [3]:
stream.to_dataframe().head()

,event_id,event_type,event_index,event,timestamp,user_id
0,af9fb778-2062-4a59-8426-e6637f592350,path_start,0,path_start,2019-11-01 17:59:13.273932,219483890
1,af9fb778-2062-4a59-8426-e6637f592350,raw,0,catalog,2019-11-01 17:59:13.273932,219483890
2,b922e77e-57c9-4a67-847f-4b3a64b605bd,raw,1,product1,2019-11-01 17:59:28.459271,219483890
3,63af602f-f9e6-4933-8400-a0de1e5aebf5,raw,2,cart,2019-11-01 17:59:29.502214,219483890
4,216033ae-7890-4ab0-b711-93b80bcdc7b6,raw,3,catalog,2019-11-01 17:59:32.557029,219483890


# 2. Creating a funnel

In [4]:
stream.funnel(stages=['catalog', 'cart', 'payment_done'])

## Interpreting above graph  
- There are 3611 users who reached catalog event.  
- 1924 of them reached cart event after they had already reached catalog event (perhaps, there were some other events between catalog and cart).  
- 653 out of these 1924 users followed catalog → cart → payment_done path.   
- The percentage values show the conversion rates from the previous or from the first stage.

In [5]:
# Grouping multiple events into one stage
stream.funnel(stages=['catalog', ['product1', 'product2'], 'cart', 'payment_done'])

# 3. Comparing funnel for different user cohorts

In [6]:
# funnels for two user cohorts: for January 2021 and for February 2021
stream_df = stream.to_dataframe()

# pandas.Series, indicies are user_ids, values are months when users appear
cohorts = stream_df.groupby('user_id').first()['timestamp'].dt.strftime('%Y-%m')

segment1 = cohorts[cohorts == '2020-01'].index
segment2 = cohorts[cohorts == '2020-02'].index

stream.funnel(
    stages=['catalog', ['product1', 'product2'], 'cart', 'payment_done'],
    stage_names=['catalog', 'product', 'cart', 'payment_done'],
    funnel_type='closed',
    segments=(segment1, segment2),
    segment_names=('converted', 'not_converted')
)

## Interpretation of above visualization

- January cohort has better conversion rates at each stage: 58% VS 52% for catalog → product transition, 66% VS 63% for product → cart transition, and 33% VS 25% for cart → payment_done transition.
- As a result, the overall conversion rate for catalog → payment_done for the January cohort is better (12% VS 8%). However, the observed difference might be obtained by chance. To estimate this explicitly, we need to test statistical hypothesis.